In [6]:
# -*- coding: utf-8 -*-
import xlrd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# 将根目录下的所有表格读入
def load_data(root_path):
    file_list = os.listdir(root_path)
    file_num = len(file_list)
    df = pd.DataFrame()
    for i in range(file_num):
        temp = pd.DataFrame(pd.read_excel(root_path + file_list[i]))
        df = pd.concat([df,temp], ignore_index = True)
    return df

rootdir_1 = 'E:\\备份\\训练数据汇总(8W)\\四川\\'
rootdir_2 = 'E:\\备份\\训练数据汇总(8W)\\南京\\'
df_1 = load_data(rootdir_1)
df_2 = load_data(rootdir_2)

head_1 = df_1.columns.values.tolist()
head_2 = df_2.columns.values.tolist()
interSection = [val for val in head_1 if val in head_2]
different_21 = list(set(head_2).difference(set(head_1)))
different_12 = list(set(head_1).difference(set(head_2)))

data_1 = df_1[['SCREEN_RESOLUTION_LONG', 'SCREEN_RESOLUTION_WIDTH', 'VIDEO_CLARITY', 'VIDEO_BITRATE',
            'BUFFERING_DELAY', 'VIDEO_BUFFERING_PEAK_RATE', 'VIDEO_REBUFF_NUM', 'VIDEO_REBUFF_TOTAL_TIME',
            'VIDEO_TOTAL_TIME', 'VIDEO_PLAY_TOTAL_TIME', 'VIDEO_ALL_PEAK_RATE', 'VIDEO_AVERAGE_RATE', 
            'PHONE_PLACE_STATE', 'ENVIRONMENTAL_NOISE', 'NETWORK_TYPE', 'MCC', 'USERBUFFERTIME',
            'GPS_LONGITUDE_FIVE', 'INSTAN_DOWNLOAD_SPEED', 'USER_SCORE', 'USER_NAME']]
data_2 = df_2[['SCREEN_RESOLUTION_LONG', 'SCREEN_RESOLUTION_WIDTH', 'VIDEO_CLARITY', 'VIDEO_BITRATE',
            'BUFFERING_DELAY', 'VIDEO_PEAK_RATE', 'VIDEO_KA_NUM', 'VIDEO_KA_TOTAL_TIME', 
            'VIDEO_TOTAL_TIME', 'VIDEO_PLAY_TOTAL_TIME', 'VIDEO_ALL_PEAK_RATE', 'VIDEO_AVERAGE_PEAK_RATE',
            'PHONE_ELECTRIC_START', 'PHONE_ELECTRIC_END', 'PHONE_VERSION', 'PING_AVG_RTT', 'SIGNAL_STRENGTH',
            'USER_SCORE', 'ELOADING', 'ESTALLING', 'EVMOS']]

column_name = ['SCREEN_RESOLUTION_LONG', 'SCREEN_RESOLUTION_WIDTH', 'VIDEO_CLARITY', 'VIDEO_BITRATE',
            'INITIAL_DELAY', 'INITIAL_DELAY_PEAK_RATE', 'VIDEO_STALLING_NUM', 'VIDEO_STALLING_TOTAL_TIME', 
            'VIDEO_TOTAL_TIME', 'VIDEO_PLAY_TOTAL_TIME', 'VIDEO_ALL_PEAK_RATE', 'VIDEO_AVERAGE_RATE',
            'PHONE_ELECTRIC_START', 'PHONE_ELECTRIC_END', 'PHONE_VERSION', 'PING_AVG_RTT', 'SIGNAL_STRENGTH',
            'USER_SCORE', 'ELOADING', 'ESTALLING', 'EVMOS']
data_1.columns = column_name
data_2.columns = column_name

raw_data = pd.concat([data_1,data_2], axis = 0)
raw_data = raw_data.reset_index(drop = True)

raw_data['VIDEO_CLARITY'].loc[raw_data['VIDEO_CLARITY'] == 240] = 1
raw_data['VIDEO_CLARITY'].loc[raw_data['VIDEO_CLARITY'] == 360] = 2
raw_data['VIDEO_CLARITY'].loc[raw_data['VIDEO_CLARITY'] == 720] = 3
raw_data['VIDEO_CLARITY'].loc[raw_data['VIDEO_CLARITY'] == 1080] = 4
raw_data['VIDEO_CLARITY'].loc[raw_data['VIDEO_CLARITY'] == 800] = 0
raw_data['VIDEO_CLARITY'].loc[raw_data['VIDEO_CLARITY'] == 808] = 0
raw_data_1 = raw_data[-raw_data.VIDEO_CLARITY.isin([0])]
raw_data_1 = raw_data.reset_index(drop = True)

raw_data_1['PIXEL_NUMBER'] = raw_data_1['SCREEN_RESOLUTION_LONG'] * raw_data_1['SCREEN_RESOLUTION_WIDTH']
raw_data_1['BATTERY_LOSS'] = raw_data_1['PHONE_ELECTRIC_START'] - raw_data_1['PHONE_ELECTRIC_END']

stalling_num = raw_data_1['VIDEO_STALLING_NUM']
stalling_total_time = raw_data_1['VIDEO_STALLING_TOTAL_TIME']
mean = []
for i in range(len(stalling_num)):
    if stalling_num[i] != 0:
        mean.append(stalling_total_time[i] / stalling_num[i])
    else:
        mean.append(0)
raw_data_1['MEAN_STALLING_DURATION'] = mean
                                       
raw_data_1['ALL_DOWCLOAD_BIT'] = raw_data_1['VIDEO_PLAY_TOTAL_TIME'] * raw_data_1['VIDEO_AVERAGE_RATE']
raw_data_1['VIDEO_SIZE'] = raw_data_1['VIDEO_TOTAL_TIME'] * raw_data_1['VIDEO_BITRATE']

C:\Users\ppxx\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
raw_data_1 = raw_data_1.drop('SIGNAL_STRENGTH', 1)

In [ ]:
raw_data_1.info()

In [14]:
raw_data_1['error'] = raw_data_1['VIDEO_SIZE'] - raw_data_1['ALL_DOWCLOAD_BIT']
raw_data_1['error'].loc[raw_data_1['error'] > 0] = 1
raw_data_1['error'].loc[raw_data_1['error'] < 0] = 0
raw_data_1['error'].value_counts()

C:\Users\ppxx\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


1    56857
0    24317
Name: error, dtype: int64

In [15]:
import re

In [16]:
a = raw_data_1.loc[:,'PING_AVG_RTT']

In [17]:
for i in range(len(a)):
    if type(a[i]) == str:
        if 'ms' in a[i]:
            a[i] = a[i].strip('ms')
        if '.' in a[i]:
            a[i] = float(a[i])

C:\Users\ppxx\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ppxx\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [18]:
raw_data_1.loc[:,'PING_AVG_RTT'] = a

In [ ]:
raw_data_1.shape

In [19]:
raw_data_1 = raw_data_1.dropna()
raw_data_1 = raw_data_1.reset_index(drop = True)

In [23]:
raw_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70153 entries, 0 to 70152
Data columns (total 26 columns):
SCREEN_RESOLUTION_LONG       70153 non-null float64
SCREEN_RESOLUTION_WIDTH      70153 non-null float64
VIDEO_CLARITY                70153 non-null object
VIDEO_BITRATE                70153 non-null object
INITIAL_DELAY                70153 non-null object
INITIAL_DELAY_PEAK_RATE      70153 non-null float64
VIDEO_STALLING_NUM           70153 non-null object
VIDEO_STALLING_TOTAL_TIME    70153 non-null object
VIDEO_TOTAL_TIME             70153 non-null float64
VIDEO_PLAY_TOTAL_TIME        70153 non-null object
VIDEO_ALL_PEAK_RATE          70153 non-null object
VIDEO_AVERAGE_RATE           70153 non-null object
PHONE_ELECTRIC_START         70153 non-null object
PHONE_ELECTRIC_END           70153 non-null object
PHONE_VERSION                70153 non-null object
PING_AVG_RTT                 70153 non-null object
USER_SCORE                   70153 non-null float64
ELOADING           

In [201]:
phone = raw_data_1['PHONE_VERSION'].tolist()

In [216]:
phone_name = ['IPHONE', 'XIAOMI', 'HUAWEI', 'HONOR', 'VIVO', 'OPPO', 
              'LENOVO', 'COOLPAD','SAMSUNG', 'MEIZU', 'LE','GIONEE' ,'ONEPLUS' ,'360',
              'K-TOUCH', 'KOPO', 'NOKIA', 'SONY','NUBIA', 'SMARTISAN', 'MOTOROLA', 'ZTE']

In [217]:
for index, name in enumerate(phone):
    for i in range(len(phone_name)):
        if phone_name[i] in name.upper():
            phone[index] = phone_name[i]

In [218]:
raw_data_1['PHONE_VERSION'] = phone

In [220]:
raw_data_1.to_csv('E:\\备份\\训练数据汇总(8W)\\409.csv')

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()

In [71]:
X_train = vec.fit_transform(phone_num).toarray()

In [74]:
X_train.shape

(70153, 312)

In [41]:
array = np.array(raw_data_1, dtype = np.float)
dataset = pd.DataFrame(array, columns = column_name)

ValueError: could not convert string to float: 'HUAWEITRT-AL00'

In [ ]:
import seaborn as sns

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(20, 20))
sns.heatmap(raw_data_1.corr(), linewidth=0.1, vmax=1.0, square=True, cmap=colormap, linecolor='white', 
            annot=True)
plt.show()